In [1]:
import pika

from billiard import current_process
import time
import math
import socket
import numpy as np
import os

pika.__version__


'1.1.0'

In [2]:
credentials = pika.PlainCredentials('myguest', 'myguestpwd')
exchangeName = 'ping'
#URL='localhost'
URL='PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com'
parameters =  pika.ConnectionParameters(URL, port=5672, credentials=credentials)
connection = pika.BlockingConnection(parameters)
channel = connection.channel()

channel.exchange_declare(exchange=exchangeName, exchange_type='fanout')
#channel.exchange_declare(exchange = exchangeName, exchange_type = "fanout", auto_delete = True)

<METHOD(['channel_number=1', 'frame_type=1', 'method=<Exchange.DeclareOk>'])>

In [3]:
def callback(ch, method, props, body):
    
    print(" [x] %r" % body)
    
    print('body:', body)
    print('ch:', ch)
    print('method:', method)
    print('props:', props)
    
    print('props.reply_to:', props.reply_to)
    print('props.correlation_id:', props.correlation_id)
    
    
    process_name=current_process()._name
    try:
        process_index=str(current_process().index)
    except:
        process_index='_none_'
    
    tmpip= socket.gethostbyname(socket.gethostname())
    
    results=tmpip+' process_name: '+process_name+' process_index: '+process_index+' os_pid: '+str(os.getpid())+' '+'**'
    print(results)
    
    mybody=str('Response from worker: '+results)
    
    # We need to send back the results to the client
    ch.basic_publish(exchange='', # Default exchange
                     routing_key='Reply', # This is the routing key to the queue that we use for sending the answer
                     #use the same correlation_id so the client knows that this is a reply to what message
                     properties=pika.BasicProperties(correlation_id = props.correlation_id), 
                     body=mybody)
    print(" Results sent back to the server")
    print(" *** \n")


In [4]:
#declare exchange and queue, bind them and consume messages
#for fanout type exchange, routing key is useless
 
#result=channel.queue_declare(queue = queueName, exclusive = True, auto_delete = True)
result = channel.queue_declare('', exclusive=True)
queueName = result.method.queue
print(queueName)

channel.queue_bind(exchange = exchangeName, queue = queueName, routing_key = None)

amq.gen-nb9coprYp33EVzYkbzqHMA


<METHOD(['channel_number=1', 'frame_type=1', 'method=<Queue.BindOk>'])>

In [ ]:
print(' [*] Waiting for logs. To exit press CTRL+C')

channel.basic_consume(queue=queueName,  on_message_callback=callback, auto_ack=True)

channel.start_consuming()

 [*] Waiting for logs. To exit press CTRL+C
 [x] b'Ping EveryOne!'
body: b'Ping EveryOne!'
ch: <BlockingChannel impl=<Channel number=1 OPEN conn=<SelectConnection OPEN transport=<pika.adapters.utils.io_services_utils._AsyncPlaintextTransport object at 0x7fd9ec01d4a8> params=<ConnectionParameters host=PROD-JOB-844fd7d2202ac4da.elb.us-east-2.amazonaws.com port=5672 virtual_host=/ ssl=False>>>>
method: <Basic.Deliver(['consumer_tag=ctag1.07aad9ce601946818a378a074859d79e', 'delivery_tag=1', 'exchange=ping', 'redelivered=False', 'routing_key='])>
props: <BasicProperties(['reply_to=amq.gen-JlwenA-ZMVn2o18wljndww'])>
props.reply_to: amq.gen-JlwenA-ZMVn2o18wljndww
props.correlation_id: None
172.31.13.51 process_name: MainProcess process_index: _none_ os_pid: 20985 **
 Results sent back to the server
 *** 

